In [1]:
from bs4 import BeautifulSoup as bs
import grequests
import requests
import pandas as pd

imo = requests.get('https://www.imovirtual.com/arrendar/apartamento/') # request http
raw_html = imo.text # convert webpage code into raw text
soup = bs(raw_html) # soupify page text

# find number of pages
for i in soup.find_all('ul', class_='pager'):
    pages = int(i.text.split()[-1])
    
# initialize url list with first page for subsequent appending 
urls = ['https://www.imovirtual.com/arrendar/apartamento/'] 

# append further pages links
for page in range(2,pages+1):
    urls.append('https://www.imovirtual.com/arrendar/apartamento/?page=' + str(page))
    
# initialize lists for house prices, types, locations and sizes
prices = []
types = []
location = []
sizes = [] 

for u in urls:
    imo = requests.get(u) # request http
    raw_html = imo.text # convert webpage code into raw text
    soup = bs(raw_html) # soupify page text

    # Find the desired data: price, location (concelho), typology and size (m2)

    # price
    for price in soup.find_all('li', class_="offer-item-price"): 
        prices.append(price.string.split('€')[0].replace(' ','').strip())

    #type 
    for ty in soup.find_all('li', class_="offer-item-rooms hidden-xs"):
        types.append(ty.string)

    #location
    for loc in soup.find_all('p', class_="text-nowrap"):
        location.append(loc.text.split('Apartamento para arrendar: ')[1])

    #size (m2)
    for size in soup.find_all('li', class_="hidden-xs offer-item-area"):
        sizes.append(size.next.split(' ')[0])       
        
all_houses = [] # list where all houses will be
all_houses.append([prices, types, location, sizes])
columns = ['price','type','location','size'] # columns names for the dataframe
all_data = pd.DataFrame(all_houses[0]) # create datafame with the list created
all_data = all_data.transpose() # transpose dataframe
all_data.columns = columns # rename dataframe columns
all_data = all_data.drop_duplicates() # drop duplicate columns
all_data = all_data[all_data['price'] != 'Preçosobconsulta'] # drop houses with negotiable prices

In [2]:
# replace house types with numeric values       
type_dict = {'T2':2, 'T3':3, 'T1':1, 'T4':4, 'T0':0, 'T5':5, 'T6':6, 'T8':8, 'T7':7, 'T10 ou superior':10, 'T9':9}
all_data['type'] = all_data['type'].replace(type_dict)

# split location into municipality and district columns
municipality = []
district = []
for i in all_data.location.str.split(', '):
    try:
        print(i[-2])
        municipality.append(i[-2])
    except:
        municipality.append(i[-1])
        
    try:
        print(i[-1])
        district.append(i[-1])
    except:
        district.append('')

all_data['municipality'] = municipality
all_data['district'] = district
all_data = all_data.drop(columns='location', axis=1)

#change size column to float data type
all_data['size'] = all_data['size'].astype('float')

# save data to local csv file
all_data.to_csv('house_data.csv')


São Vicente
Lisboa
Santo António
Lisboa
Arroios
Lisboa
Odivelas
Lisboa
Estrela
Lisboa
Cascais
Lisboa
Vila Nova de Gaia
Porto
São Nicolau e Vitória
Porto
Avenidas Novas
Lisboa
Misericórdia
Lisboa
Santa Maria Maior
Lisboa
Belém
Lisboa
Campolide
Lisboa
Santo António
Lisboa
Estrela
Lisboa
Misericórdia
Lisboa
Amadora
Lisboa
Santa Maria Maior
Lisboa
Santo António
Lisboa
Santo António
Lisboa
Avenidas Novas
Lisboa
Cascais
Lisboa
Alvalade
Lisboa
Santa Maria Maior
Lisboa
Avenidas Novas
Lisboa
Avenidas Novas
Lisboa
Avenidas Novas
Lisboa
Avenidas Novas
Lisboa
Santo António
Lisboa
Santo António
Lisboa
Santo António
Lisboa
Alcântara
Lisboa
Alcântara
Lisboa
Santo António
Lisboa
Avenidas Novas
Lisboa
Santo António
Lisboa
Avenidas Novas
Lisboa
Avenidas Novas
Lisboa
Santo António
Lisboa
Arroios
Lisboa
Avenidas Novas
Lisboa
Santo António
Lisboa
Misericórdia
Lisboa
Belém
Lisboa
Santo António
Lisboa
Avenidas Novas
Lisboa
Santo António
Lisboa
Cascais
Lisboa
Estrela
Lisboa
Cascais
Lisboa
Avenidas Novas
Lisbo

Cascais
Lisboa
Campolide
Lisboa
Cascais
Lisboa
Olivais
Lisboa
São Nicolau e Vitória
Porto
Areeiro
Lisboa
Amadora
Lisboa
Estrela
Lisboa
Amadora
Lisboa
São Domingos de Benfica
Lisboa
Alvalade
Lisboa
Misericórdia
Lisboa
Penha de França
Lisboa
Oeiras
Lisboa
União de Freguesias da cidade de Santarém
Santarém
Santa Maria Maior
Lisboa
Campolide
Lisboa
Santo António
Lisboa
Santo António dos Olivais
Coimbra
Penha de França
Lisboa
Penha de França
Lisboa
Santo António dos Olivais
Coimbra
Penha de França
Lisboa
Benfica
Lisboa
Amadora
Lisboa
Arroios
Lisboa
São Vicente
Lisboa
Barreiro
Setúbal
Estrela
Lisboa
São Nicolau e Vitória
Porto
Seixal
Setúbal
Cascais
Lisboa
Nossa Senhora da Anunciada e Santa Maria da Graça)
Setúbal
Estrela
Lisboa
Avenidas Novas
Lisboa
Glória e Vera Cruz
Aveiro
Seixal
Setúbal
Oeiras
Lisboa
Misericórdia
Lisboa
Sintra
Lisboa
Campo de Ourique
Lisboa
Matosinhos
Porto
Estrela
Lisboa
Santo António
Lisboa
Nossa Senhora da Anunciada e Santa Maria da Graça)
Setúbal
Fraião e Lamaçães
Br

Porto
Santo António
Lisboa
Vila Nova de Gaia
Porto
Glória e Vera Cruz
Aveiro
São Domingos de Benfica
Lisboa
Odivelas
Lisboa
Amadora
Lisboa
Odivelas
Lisboa
Lumiar
Lisboa
Oeiras
Lisboa
Odivelas
Lisboa
Lordelo do Ouro e Massarelos
Porto
Misericórdia
Lisboa
Paranhos
Porto
São Vicente
Lisboa
Sintra
Lisboa
Bonfim
Porto
Alvalade
Lisboa
São Domingos de Benfica
Lisboa
Sintra
Lisboa
Santa Clara
Lisboa
Penha de França
Lisboa
Arroios
Lisboa
Seixal
Setúbal
Seixal
Setúbal
Estrela
Lisboa
Parque das Nações
Lisboa
Campolide
Lisboa
Cascais
Lisboa
Santa Maria Maior
Lisboa
São Nicolau e Vitória
Porto
Areeiro
Lisboa
Estrela
Lisboa
Penha de França
Lisboa
Misericórdia
Lisboa
São Nicolau e Vitória
Porto
Cascais
Lisboa
Penha de França
Lisboa
Lordelo do Ouro e Massarelos
Porto
São Domingos de Benfica
Lisboa
Campanhã
Porto
Ajuda
Lisboa
São Nicolau e Vitória
Porto
Santa Maria Maior
Lisboa
Entroncamento
Santarém
Barreiro
Setúbal
Lumiar
Lisboa
Benfica
Lisboa
Loures
Lisboa
Sintra
Lisboa
Odivelas
Lisboa
Odivelas
Lisb

Santo António
Lisboa
Cascais
Lisboa
Santo António
Lisboa
Santa Maria Maior
Lisboa
Santa Clara
Lisboa
Arroios
Lisboa
Arroios
Lisboa
Campo de Ourique
Lisboa
Foz do Douro e Nevogilde
Porto
Penha de França
Lisboa
Misericórdia
Lisboa
Carnide
Lisboa
Areeiro
Lisboa
Bonfim
Porto
Santo António
Lisboa
Santa Maria Maior
Lisboa
Penha de França
Lisboa
Misericórdia
Lisboa
Matosinhos
Porto
Lumiar
Lisboa
Matosinhos
Porto
Barreiro
Setúbal
Avenidas Novas
Lisboa
Estrela
Lisboa
Parque das Nações
Lisboa
Vila Nova de Gaia
Porto
São Nicolau e Vitória
Porto
Misericórdia
Lisboa
Matosinhos
Porto
Santa Clara
Lisboa
Vila Nova de Gaia
Porto
Vila Nova de Gaia
Porto
Ramalde
Porto
Arroios
Lisboa
Paranhos
Porto
Paranhos
Porto
Santa Joana
Aveiro
Odivelas
Lisboa
Lordelo do Ouro e Massarelos
Porto
Cascais
Lisboa
Almedina e São Bartolomeu)
Coimbra
Lordelo do Ouro e Massarelos
Porto
Parque das Nações
Lisboa
Barreiro
Setúbal
Entroncamento
Santarém
Lordelo do Ouro e Massarelos
Porto
Foz do Douro e Nevogilde
Porto
Santa Maria

Nossa Senhora da Anunciada e Santa Maria da Graça)
Setúbal
Misericórdia
Lisboa
Misericórdia
Lisboa
Santa Clara
Lisboa
Foz do Douro e Nevogilde
Porto
Oeiras
Lisboa
Braga (São José de São Lázaro e São João do Souto)
Braga
São Victor
Braga
Lordelo do Ouro e Massarelos
Porto
Paranhos
Porto
Ramalde
Porto
Matosinhos
Porto
Bonfim
Porto
Matosinhos
Porto
Glória e Vera Cruz
Aveiro
Paranhos
Porto
Cascais
Lisboa
Covilhã
Castelo Branco
Valongo
Porto
Campo de Ourique
Lisboa
Parque das Nações
Lisboa
Alvalade
Lisboa
Santo António
Lisboa
Vila Nova de Gaia
Porto
Loures
Lisboa
Avenidas Novas
Lisboa
Esgueira
Aveiro
Misericórdia
Lisboa
Seixal
Setúbal
Benfica
Lisboa
Vila Nova de Gaia
Porto
Braga (São José de São Lázaro e São João do Souto)
Braga
Vila Nova de Gaia
Porto
Penha de França
Lisboa
Misericórdia
Lisboa
Vila Nova de Gaia
Porto
Estarreja
Aveiro
São Domingos de Benfica
Lisboa
Matosinhos
Porto
São Nicolau e Vitória
Porto
Vila do Conde
Porto
Santo António
Lisboa
Misericórdia
Lisboa
São Victor
Braga
Sé e

...

Lisboa
Alcântara
Lisboa
Barreiro
Setúbal
Torres Vedras
Lisboa
Gondomar
Porto
Ajuda
Lisboa
Benfica
Lisboa
Estremoz
Évora
Misericórdia
Lisboa
Amadora
Lisboa
Oeiras
Lisboa
São Nicolau e Vitória
Porto
Oeiras
Lisboa
Matosinhos
Porto
Benfica
Lisboa
Vila Franca de Xira
Lisboa
São Nicolau e Vitória
Porto
Oeiras
Lisboa
Loures
Lisboa
Oeiras
Lisboa
Portimão
Faro
Glória e Vera Cruz
Aveiro
Santa Maria da Feira
Aveiro
Cascais
Lisboa
Glória e Vera Cruz
Aveiro
Belém
Lisboa
Santa Maria Maior
Lisboa
Paranhos
Porto
Vila Franca de Xira
Lisboa
Cascais
Lisboa
Valongo
Porto
Beato
Lisboa
Santa Cruz
Ilha da Madeira
Vila do Conde
Porto
Avenidas Novas
Lisboa
Cascais
Lisboa
Lordelo do Ouro e Massarelos
Porto
Bonfim
Porto
Bonfim
Porto
Santa Bárbara de Nexe
Faro
São Vicente
Braga
Portimão
Faro
Seixal
Setúbal
Loures
Lisboa
São Victor
Braga
Nossa Senhora da Anunciada e Santa Maria da Graça)
Setúbal
Portimão
Faro
Cascais
Lisboa
Loures
Lisboa
Portimão
Faro
Maia
Porto
Areeiro
Lisboa
Sé e Cividade)
Braga
Cascais
Lisboa
L

Lisboa
Marinha Grande
Leiria
Almada
Setúbal
Viseu
Viseu
Povoa de Varzim
Porto
Sé e Cividade)
Braga
Campanhã
Porto
Avenidas Novas
Lisboa
Matosinhos
Porto
Braga (São José de São Lázaro e São João do Souto)
Braga
Alvalade
Lisboa
Guimarães
Braga
São Vicente
Lisboa
Amadora
Lisboa
Amadora
Lisboa
Barreiro
Setúbal
Olivais
Lisboa
Arroios
Lisboa
Braga (São José de São Lázaro e São João do Souto)
Braga
São Vicente
Braga
Bonfim
Porto
Braga (São José de São Lázaro e São João do Souto)
Braga
Odivelas
Lisboa
Palmela
Setúbal
Lumiar
Lisboa
Ponta do Sol
Ilha da Madeira
São Vicente
Braga
Areosa
Viana do Castelo
Bonfim
Porto
Ramalde
Porto
Ramalde
Porto
Bonfim
Porto
Paranhos
Porto
Esposende
Braga
Oeiras
Lisboa
Faro (Sé e São Pedro)
Faro
Santa Maria Maior
Lisboa
Amadora
Lisboa
Cascais
Lisboa
Oeiras
Lisboa
Oeiras
Lisboa
Oeiras
Lisboa
Benfica
Lisboa
Foz do Douro e Nevogilde
Porto
Santa Maria Maior
Lisboa
Odivelas
Lisboa
Funchal
Ilha da Madeira
Santo António dos Olivais
Coimbra
Santa Maria Maior
Lisboa
Penha d

Porto
Palmeira
Braga
Montijo
Setúbal
Vila Nova de Gaia
Porto
Povoa de Varzim
Porto
Caldas da Rainha
Leiria
Santa Maria Maior
Lisboa
Vila Nova de Gaia
Porto
Paranhos
Porto
Vila Nova de Gaia
Porto
Barreira e Cortes
Leiria
Oeiras
Lisboa
Paranhos
Porto
Ourém
Santarém
São Nicolau e Vitória
Porto
Misericórdia
Lisboa
Almedina e São Bartolomeu)
Coimbra
Nogueiró e Tenões
Braga
Glória e Vera Cruz
Aveiro
São Victor
Braga
Campolide
Lisboa
Olivais
Lisboa
Vila Nova de Gaia
Porto
Vila Nova de Gaia
Porto
Cascais
Lisboa
Santo António dos Olivais
Coimbra
Glória e Vera Cruz
Aveiro
Santo António dos Olivais
Coimbra
União de Freguesias da cidade de Santarém
Santarém
Santa Clara e Castelo Viegas
Coimbra
Loures
Lisboa
Santa Clara e Castelo Viegas
Coimbra
Ramalde
Porto
Guimarães
Braga
Santo António dos Olivais
Coimbra
Covilhã
Castelo Branco
Campo de Ourique
Lisboa
Misericórdia
Lisboa
Avenidas Novas
Lisboa
Lumiar
Lisboa
Viana do Castelo (Santa Maria Maior e Monserrate) e Meadela
Viana do Castelo
Oliveira de Az

Paredes
Porto
São Nicolau e Vitória
Porto
Odivelas
Lisboa
Almada
Setúbal
Vila Real de Santo António
Faro
Lumiar
Lisboa
Paranhos
Porto
Lordelo do Ouro e Massarelos
Porto
Parque das Nações
Lisboa
Campo de Ourique
Lisboa
São Nicolau e Vitória
Porto
Figueira da Foz
Coimbra
Cascais
Lisboa
Maia
Porto
Santa Maria Maior
Lisboa
Ajuda
Lisboa
Barreiro
Setúbal
Olivais
Lisboa
Bonfim
Porto
Bonfim
Porto
Funchal
Ilha da Madeira
Vila Nova de Gaia
Porto
Santo António
Lisboa
Seixal
Setúbal
Vila Nova de Gaia
Porto
São Victor
Braga
Benavente
Santarém
Ajuda
Lisboa
Foz do Douro e Nevogilde
Porto
Loures
Lisboa
Ponta Delgada
Ilha de São Miguel
Odivelas
Lisboa
Oeiras
Lisboa
Santa Maria Maior
Lisboa
Seixal
Setúbal
Lordelo do Ouro e Massarelos
Porto
Amadora
Lisboa
Arroios
Lisboa
Campolide
Lisboa
Gondomar
Porto
Glória e Vera Cruz
Aveiro
Lordelo do Ouro e Massarelos
Porto
Funchal
Ilha da Madeira
Foz do Douro e Nevogilde
Porto
Carnide
Lisboa
Lordelo do Ouro e Massarelos
Porto
Fraião e Lamaçães
Braga
Amadora
Lisboa
F

ValueError: could not convert string to float: '118,25'

In [3]:
all_data

,price,type,size,municipality,district
0,2100,6,220,São Vicente,Lisboa
1,990,3,70,Santo António,Lisboa
2,750,2,27,Arroios,Lisboa
3,1200,4,146,Odivelas,Lisboa
4,1850,4,140,Estrela,Lisboa
...,...,...,...,...,...
11995,600,3,122,Ílhavo,Aveiro
11996,425,1,30,Alenquer,Lisboa
11997,1700,1,75,São Domingos de Benfica,Lisboa
11998,1500,3,140,Olivais,Lisboa


In [4]:
all_data.dtypes

price           object
type             int64
size            object
municipality    object
district        object
dtype: object